In [ ]:
# Author : Tapas

## Applications of Machine Learning in Stock Markets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.Data Preparation

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from pandas.plotting import lag_plot

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
today = dt.date.today()
year_back = 20
end_date = today - dt.timedelta(days = 365 * year_back)

In [12]:
print(today)
print(end_date)

2023-03-29
2003-04-03


In [ ]:
# reliance_data = pdr.get_data_yahoo(f"{'RELIANCE.NS'}",today,end_date)
# reliance_data.sample(5)

In [ ]:
# # download dataframe
# data = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-04-30")
# data

In [ ]:
# df= yf.Ticker("^NSEBANK").history(period='12y').reset_index()
# df_axis= yf.Ticker("AXISBANK.NS").history(period='3y').reset_index()
# df_sbi= yf.Ticker("SBIN.NS").history(period='3y').reset_index()
# df_rbl= yf.Ticker("RBLBANK.NS").history(period='3y').reset_index()
# df_pnb= yf.Ticker("PNB.NS").history(period='3y').reset_index()
# df_kot= yf.Ticker("KOTAKBANK.NS").history(period='3y').reset_index()
# df_ind= yf.Ticker("INDUSINDBK.NS").history(period='3y').reset_index()
# df_idfc= yf.Ticker("IDFCFIRSTB.NS").history(period='3y').reset_index()
# df_icic= yf.Ticker("ICICIBANK.NS").history(period='3y').reset_index()
# df_band= yf.Ticker("BANDHANBNK.NS").history(period='3y').reset_index()
# df_hdfc= yf.Ticker("HDFC.NS").history(period='3y').reset_index()
# df_fed= yf.Ticker("FEDERALBNK.NS").history(period='3y').reset_index()
# df_au= yf.Ticker("AUBANK.NS").history(period='3y').reset_index()

In [ ]:
# df= yf.download(tickers = "^NSEBANK",  # list of tickers
#             period = "20y",         # time period
#             interval = "1d",       # trading interval
#             ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
#             prepost = False       # download pre/post market hours data?
#             ).reset_index()

In [ ]:
# df= yf.download(tickers = "^NSEBANK",
#             start = '2008-01-01', end = today,
#             interval = "1d",
#             ignore_tz = True,
#             prepost = False
#             ).reset_index()

# df= yf.download(tickers = "^NSEBANK",
#             period = "max",
#             interval = "1d",
#             ignore_tz = True,
#             prepost = False
#             ).reset_index()

In [4]:
df= yf.download(tickers = "^NSEBANK",
            start = '2008-01-01',
            interval = "1d",
            ignore_tz = True,
            prepost = False
            ).reset_index()

[*********************100%***********************]  1 of 1 completed


In [14]:
df.tail(15)

,Date,Open,High,Low,Close,Adj Close,Volume
3450,2023-03-09,41532.648438,41659.000000,41208.101562,41256.750000,41256.750000,119200
3451,2023-03-10,40805.250000,40839.000000,40341.699219,40485.449219,40485.449219,145300
3452,2023-03-13,40356.101562,40690.050781,39454.601562,39564.699219,39564.699219,174500
3453,2023-03-14,39522.398438,39768.500000,39132.601562,39411.398438,39411.398438,205700
3454,2023-03-15,39777.898438,39914.101562,38934.648438,39051.500000,39051.500000,164300
3455,2023-03-16,39061.699219,39381.398438,38613.148438,39132.601562,39132.601562,196300
3456,2023-03-17,39442.398438,39705.148438,38926.800781,39598.101562,39598.101562,173400
3457,2023-03-20,39512.101562,39512.101562,38941.898438,39361.949219,39361.949219,173800
3458,2023-03-21,39599.648438,39970.898438,39366.300781,39894.699219,39894.699219,187900
3459,2023-03-22,40036.000000,40085.601562,39837.800781,39999.050781,39999.050781,0


In [5]:
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d").dt.floor('D')
display(df['Date'].min())
display(df['Date'].max())

Timestamp('2008-01-01 00:00:00')

Timestamp('2023-03-29 00:00:00')

In [6]:
max_date = str(df['Date'].max())
dateprocess_max = dt.datetime.strptime(max_date, "%Y-%m-%d %H:%M:%S")
min_date = str(df['Date'].min())
dateprocess_min = dt.datetime.strptime(min_date, "%Y-%m-%d %H:%M:%S")
#max_date_limit = (dateprocess_max - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
min_date_limit = str(dateprocess_min.year) +'-'+ str(dateprocess_min.month) +'-'+ str(dateprocess_min.day)
max_date_limit = str(dateprocess_max.year) +'-'+ str(dateprocess_max.month) +'-'+ str(dateprocess_max.day)

In [7]:
dates_required = pd.date_range(min_date, max_date, freq='D').strftime("%Y-%m-%d").tolist()
dates_required = pd.DataFrame(dates_required, columns = ['Date'])
dates_required.Date = pd.to_datetime(dates_required.Date)
processed_data = dates_required.merge(df,on='Date', how='left')
processed_data.columns = df.columns
processed_data = processed_data.fillna(method='ffill')
processed_data['Date'] = pd.to_datetime(processed_data['Date'], format="%Y-%m-%d").dt.floor('D')

In [ ]:
processed_data['OCDifference'] = processed_data['Open']-processed_data['Close']
processed_data['OCDiff_perc'] = (processed_data['Open']-processed_data['Close'])/processed_data['Open']*100

processed_data['OHDifference'] = processed_data['Open']-processed_data['High']
processed_data['OHDiff_perc'] = (processed_data['Open']-processed_data['High'])/processed_data['Open']*100

processed_data['OLDifference'] = processed_data['Open']-processed_data['Low']
processed_data['OLDiff_perc'] = (processed_data['Open']-processed_data['Low'])/processed_data['Open']*100

processed_data['OADifference'] = processed_data['Open']-processed_data['Adj Close']
processed_data['OADiff_perc'] = (processed_data['Open']-processed_data['Adj Close'])/processed_data['Open']*100

In [ ]:
processed_data.tail(15)

,Date,Open,High,Low,Close,Adj Close,Volume,OCDifference,OCDiff_perc,OHDifference,OHDiff_perc,OLDifference,OLDiff_perc,OADifference,OADiff_perc
5552,2023-03-15,39777.898438,39914.101562,38934.648438,39051.500000,39051.500000,164300.0,726.398438,1.826136,-136.203125,-0.342409,843.250000,2.119896,726.398438,1.826136
5553,2023-03-16,39061.699219,39381.398438,38613.148438,39132.601562,39132.601562,196300.0,-70.902344,-0.181514,-319.699219,-0.818447,448.550781,1.148314,-70.902344,-0.181514
5554,2023-03-17,39442.398438,39705.148438,38926.800781,39598.101562,39598.101562,173400.0,-155.703125,-0.394761,-262.750000,-0.666161,515.597656,1.307217,-155.703125,-0.394761
5555,2023-03-18,39442.398438,39705.148438,38926.800781,39598.101562,39598.101562,173400.0,-155.703125,-0.394761,-262.750000,-0.666161,515.597656,1.307217,-155.703125,-0.394761
5556,2023-03-19,39442.398438,39705.148438,38926.800781,39598.101562,39598.101562,173400.0,-155.703125,-0.394761,-262.750000,-0.666161,515.597656,1.307217,-155.703125,-0.394761
5557,2023-03-20,39512.101562,39512.101562,38941.898438,39361.949219,39361.949219,173800.0,150.152344,0.380016,0.000000,0.000000,570.203125,1.443110,150.152344,0.380016
5558,2023-03-21,39599.648438,39970.898438,39366.300781,39894.699219,39894.699219,187900.0,-295.050781,-0.745084,-371.250000,-0.937508,233.347656,0.589267,-295.050781,-0.745084
5559,2023-03-22,40036.000000,40085.601562,39837.800781,39999.050781,39999.050781,0.0,36.949219,0.092290,-49.601562,-0.123892,198.199219,0.495052,36.949219,0.092290
5560,2023-03-23,39836.148438,40201.601562,39552.500000,39616.898438,39616.898438,153200.0,219.250000,0.550380,-365.453125,-0.917391,283.648438,0.712038,219.250000,0.550380
5561,2023-03-24,39555.250000,39767.898438,39294.898438,39395.351562,39395.351562,166100.0,159.898438,0.404241,-212.648438,-0.537599,260.351562,0.658197,159.898438,0.404241


# 2. Fit Model

## 2.1 LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(7)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
Scale=StandardScaler()
def data_prep(df, cols_str, lookback, future, Scale):
    date_train=pd.to_datetime(df['Date'], format="%Y-%m-%d").dt.floor('D')
    cols = cols_str.split(",")
    df_for_training = df[cols].astype(float)
    
    #df_for_training_scaled=Scale.fit_transform(df_train)
    scaler = Scale.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)

    trainX = []
    trainY = []
    for i in range(lookback, len(df_for_training_scaled) - future +1):
        trainX.append(df_for_training_scaled[i - lookback:i, 0:df_for_training.shape[1]])
        trainY.append(df_for_training_scaled[i + future - 1:i + future, 0])

    trainX, trainY = np.array(trainX), np.array(trainY)
        
    return trainX, trainY, df_for_training, date_train

In [ ]:
cols_str = 'Open,High,Low,Close,Adj Close,Volume,OCDifference,OCDiff_perc,OHDifference,OHDiff_perc,OLDifference,OLDiff_perc,OADifference,OADiff_perc'
lookback = 30
future = 1

In [ ]:
#Lstm_x, Lstm_y, df_train, date_train = data_prep(processed_data, cols_str, 12, 1, Scale)

In [ ]:
def Lstm_model(trainX,trainY):
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
    #model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(LSTM(50, activation='relu', return_sequences=True))
    #model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(LSTM(50, activation='relu', return_sequences=True))
    #model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(LSTM(units = 50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(trainY.shape[1]))

    ## compile the model and specify loss and optimizer
    #opt = keras.optimizers.Adam(learning_rate=0.01)
    #opt = Adam(learning_rate=0.01, beta_1=0.85, beta_2=0.999)
    #model.compile(optimizer=opt, loss='mse')

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.summary()
    
    # fit the model
    es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
    model.fit(trainX, trainY, epochs=100, batch_size=64, validation_split=0.1, verbose=1, callbacks=[es])
    #model.fit(trainX, trainY, epochs=100, batch_size=24, validation_split=0.1, verbose=1)
    
    return model

In [ ]:
# def Lstm_model1(X, y):
#     regressor = Sequential()
#     regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
#     regressor.add(Dropout(0.2))
#     regressor.add(LSTM(units = 50, return_sequences = True))
#     regressor.add(Dropout(0.2))
#     regressor.add(LSTM(units = 50, return_sequences = True))
#     regressor.add(Dropout(0.2))
#     regressor.add(LSTM(units = 50))
#     regressor.add(Dropout(0.2))
#     regressor.add(Dense(units = 1))

#     regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
#     es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
#     regressor.fit(X, y, epochs = 100, validation_split=0.1, batch_size = 64, verbose=1, callbacks=[es])
#     return regressor

In [ ]:
def predict_open(model, date_train, trainX, df_train, future, Scale):
    forecasting_dates=pd.date_range(list(date_train)[-1], periods=future +1, freq='D').tolist()
    #from pandas.tseries.offsets import DateOffset
    #forecasting_dates = [df.Date[-1:] + DateOffset(months=x) for x in range(0, future + 1)]
    forecasting_dates.pop(0)
    
    predicted = model.predict(trainX[-future:])
    prediction_copies = np.repeat(predicted, df_train.shape[1], axis=-1)
    predicted_descaled = Scale.inverse_transform(prediction_copies)[:,0]
    
    return predicted_descaled,forecasting_dates

def output_prep(forecasting_dates, predicted_descaled, pred_col_name):
    # Convert timestamp to date
    forecast_dates = []
    for time_i in forecasting_dates:
        forecast_dates.append(time_i.date())
    
    y_fcst_lower = predicted_descaled * 0.999
    y_fcst_upper = predicted_descaled * 1.002
    low_col = pred_col_name + '_lower'
    up_col = pred_col_name + '_upper'
    #pred_col_name = 'Frcst_'+ pred_col_name
    df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), low_col:y_fcst_lower,
                                pred_col_name:predicted_descaled, up_col:y_fcst_upper})
    df_forecast['Date']=pd.to_datetime(df_forecast['Date']).dt.floor('D')
    df_forecast = df_forecast.round(2)
    
    return df_forecast


In [ ]:
def results(df, lookback, future, Scale, cols, pred_col_name, plot_title, n_pred_future):
    Lstm_x, Lstm_y, df_train, date_train = data_prep(df, cols, lookback, future, Scale)
    model=Lstm_model(Lstm_x,Lstm_y)
    loss=pd.DataFrame(model.history.history)
    loss.plot()
    predicted_descaled,forecasting_dates = predict_open(model, date_train, Lstm_x, df_train, n_pred_future, Scale)
    results = output_prep(forecasting_dates, predicted_descaled, pred_col_name)   
    print(results.head())
    plt.show()
    fig = px.area(results, x="Date", y=pred_col_name, title=plot_title)
    fig.update_yaxes(range=[results[pred_col_name].min()-10, results[pred_col_name].max()+10])
    fig.show()
    
    return results

In [ ]:
cols_str = 'Open,High,Low,Close,Adj Close,Volume,OCDifference,OCDiff_perc,OHDifference,OHDiff_perc,OLDifference,OLDiff_perc,OADifference,OADiff_perc'
lookback = 30
future = 1
pred_col_name = 'F_Open'
plot_title = 'Open value of BankNifty'
n_pred_future = 30

df_forecast = results(processed_data, lookback, future, Scale, cols_str, pred_col_name, plot_title, n_pred_future)
# df_forecast = pd.DataFrame()
# for prod in df.Item_Code.unique():
#     print(prod)
#     df1 = df[df['Item_Code']==prod]
#     res = results(df1, lookback, future, Scale, cols_str, pred_col_name, plot_title, n_pred_future)
#     df_forecast = pd.concat([df_forecast,res])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            20224     
                                                                 
 lstm_1 (LSTM)               (None, 30, 50)            23000     
                                                                 
 lstm_2 (LSTM)               (None, 30, 50)            20200     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 83,675
Trainable params: 83,675
Non-traina

In [33]:
df_forecast.head()

,Date,F_Open_lower,F_Open,F_Open_upper
0,2023-03-30,38589.121094,38627.761719,38705.011719
1,2023-03-31,38569.511719,38608.121094,38685.328125
2,2023-04-01,38674.761719,38713.480469,38790.898438
3,2023-04-02,38655.261719,38693.960938,38771.351562
4,2023-04-03,39088.351562,39127.480469,39205.730469
5,2023-04-04,39183.308594,39222.531250,39300.980469
6,2023-04-05,39323.781250,39363.140625,39441.871094
7,2023-04-06,39231.839844,39271.109375,39349.660156
8,2023-04-07,39255.589844,39294.878906,39373.468750
9,2023-04-08,39508.960938,39548.511719,39627.601562


In [ ]:
# # save the scaler
# import pickle
# pickle.dump(Scale, open('/content/drive/MyDrive/Colab Notebooks/Scale.pkl', 'wb'))
# # load the scaler
# Scale = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Scale.pkl', 'rb'))

In [ ]:
# # Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
# model.save("/content/drive/MyDrive/Colab Notebooks/my_h5_model.h5")
# # It can be used to reconstruct the model identically.
# saved_model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/my_h5_model.h5")

In [34]:
import os
os.getcwd()

'/content'

## 2.2 FBPROPHET

In [43]:
# !pip install --user kats
# !pip install pystan==2.19.1.1
# !pip install fbprophet==0.7.1

In [22]:
# n_pred_future = 30
# from kats.consts import TimeSeriesData as tsd
# data = tsd(processed_data[['Date','Open']])
# from kats.models.prophet import ProphetModel
# from kats.models.prophet import ProphetParams

# parms = ProphetParams(seasonality_mode='multiplicative', interval_width=0.95)
# model = ProphetModel(data,parms)
# model.fit()
# forcast = model.predict(steps = n_pred_future, freq="D")

In [18]:
from fbprophet import Prophet

data = processed_data[['Date','Open']]
data.columns = ['ds','y']
n_pred_future = 30
#m = Prophet(seasonality_mode='multiplicative', interval_width=0.999)
m = Prophet(seasonality_mode='multiplicative')
m.fit(data)
future = m.make_future_dataframe(n_pred_future, freq='D', include_history=False)
p_forecast = m.predict(future)
p_forecast = p_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
p_forecast['yhat_lower'] = p_forecast['yhat'] * 0.999
p_forecast['yhat_upper'] = p_forecast['yhat'] * 1.002
p_forecast.rename(columns = {'ds':'Date','yhat':'F_Open','yhat_lower':'F_Open_lower','yhat_upper':'F_Open_upper'}, inplace = True)
p_forecast = p_forecast.round(2)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [19]:
p_forecast.head()

,ds,yhat,yhat_lower,yhat_upper
0,2023-03-30,41791.94,39456.31,43857.36
1,2023-03-31,41789.06,39499.38,43955.23
2,2023-04-01,41814.77,39412.66,43922.06
3,2023-04-02,41845.51,39683.67,44090.55
4,2023-04-03,41887.43,39629.51,44114.11
5,2023-04-04,41948.27,39577.51,44258.72
6,2023-04-05,42052.00,39764.03,44357.03
7,2023-04-06,42132.28,39849.17,44541.57
8,2023-04-07,42157.85,39784.14,44333.01
9,2023-04-08,42208.93,40021.40,44540.26


# 3. Dump forcast results

In [2]:
today = dt.date.today()
currentMonth = today.strftime("%b")

In [36]:
path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}'
isExist = os.path.exists(path)
if not isExist:
  # Create a new directory because it does not exist
  os.makedirs(path)

In [37]:
# for LSTM
lstm_path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}/BankNifty_{currentMonth}_lstm.xlsx'
df_forecast.to_excel(lstm_path, index=False)

In [ ]:
# for Prophet
prophet_path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}/BankNifty_{currentMonth}_prophet.xlsx'
p_forecast.to_excel(prophet_path, index=False)

# 4. Checking anamoly

## 4.1 LSTM

In [83]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf

In [84]:
today = dt.date.today()
currentMonth = today.strftime("%b")

In [87]:
lstm_path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}/BankNifty_{currentMonth}_lstm.xlsx'
df = pd.read_excel(lstm_path)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.floor('D')
df = df[df['Date']==today.strftime('%Y-%m-%d')]

In [88]:
df

,Date,F_Open_lower,F_Open,F_Open_upper
1,2023-03-31,38569.511719,38608.121094,38685.328125


In [89]:
# read the result file
result_path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}/BankNifty_{currentMonth}_result.xlsx'
isExist = os.path.exists(result_path)
if not isExist:
  result = pd.DataFrame()
else :
  result = pd.read_excel(result_path)

In [91]:
df1= yf.download(tickers = "^NSEBANK",
            start = today,
            interval = "1d",
            ignore_tz = True,
            prepost = False
            ).reset_index()
df1 = df1[['Date','Open']]
df1

[*********************100%***********************]  1 of 1 completed


,Date,Open
0,2023-03-31,40231.25


In [98]:
today_result = pd.DataFrame()
today_result['Date'] = df1['Date']
today_result['Open'] = df1['Open']
today_result['F_Open'] = df['F_Open'].values
today_result['F_Open_lower'] = df['F_Open_lower'].values
today_result['F_Open_upper'] = df['F_Open_upper'].values
today_result['Date'] = pd.to_datetime(today_result['Date'], format='%Y-%m-%d').dt.floor('D')
today_result['Comment'] = np.where(
    today_result['Open'] < today_result['F_Open_lower'], 'below lower level',
    np.where(
        (today_result['F_Open_lower'] <= today_result['Open']) & (today_result['Open'] < today_result['F_Open']), 
        'between low and open level',
        np.where(
            (today_result['F_Open'] <= today_result['Open']) & (today_result['Open'] < today_result['F_Open_upper']), 
            'between open and high level', 
            'above high'
        )
    )
)
today_result['F_Method'] = 'lstm'

In [99]:
today_result

,Date,Open,F_Open,F_Open_lower,F_Open_upper,Comment,F_Method
0,2023-03-31,40231.25,38608.121094,38569.511719,38685.328125,above high,lstm


In [100]:
today_result = pd.concat([result,today_result])
today_result.to_excel(result_path, index=False)

## 4.2 Prophet

In [101]:
prophet_path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}/BankNifty_{currentMonth}_prophet.xlsx'
df = pd.read_excel(prophet_path)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.floor('D')
df = df[df['Date']==today.strftime('%Y-%m-%d')]

# read the result file
result_path = f'/content/drive/MyDrive/Colab Notebooks/{currentMonth}/BankNifty_{currentMonth}_result.xlsx'
isExist = os.path.exists(result_path)
if not isExist:
  result = pd.DataFrame()
else :
  result = pd.read_excel(result_path)

df1= yf.download(tickers = "^NSEBANK",
            start = today,
            interval = "1d",
            ignore_tz = True,
            prepost = False
            ).reset_index()
df1 = df1[['Date','Open']]

today_result = pd.DataFrame()
today_result['Date'] = df1['Date']
today_result['Open'] = df1['Open']
today_result['F_Open'] = df['F_Open'].values
today_result['F_Open_lower'] = df['F_Open_lower'].values
today_result['F_Open_upper'] = df['F_Open_upper'].values
today_result['Date'] = pd.to_datetime(today_result['Date'], format='%Y-%m-%d').dt.floor('D')
today_result['Comment'] = np.where(
    today_result['Open'] < today_result['F_Open_lower'], 'below lower level',
    np.where(
        (today_result['F_Open_lower'] <= today_result['Open']) & (today_result['Open'] < today_result['F_Open']), 
        'between low and open level',
        np.where(
            (today_result['F_Open'] <= today_result['Open']) & (today_result['Open'] < today_result['F_Open_upper']), 
            'between open and high level', 
            'above high'
        )
    )
)
today_result['F_Method'] = 'prophet'
today_result = pd.concat([result,today_result])
today_result.to_excel(result_path, index=False)

[*********************100%***********************]  1 of 1 completed
